This notebook is an interactive introduction to `ucc`.

Unitary Compiler Collection (`ucc`) is a Python library for frontend-agnostic, high performance compilation of quantum circuits. UCC's goal is to gather together the best of open source compilation to make quantum programming simpler, faster, and more scalable. For more details, click the links below and read the `ucc` launch [blogpost](https://unitary.foundation/posts/2025_ucc_launch_blog/)

[![Repository](https://img.shields.io/badge/GitHub-5C5C5C.svg?logo=github)](https://github.com/unitaryfund/ucc)
[![Unitary Foundation](https://img.shields.io/badge/Supported%20By-Unitary%20Foundation-FFFF00.svg)](https://unitary.foundation)
[![Documentation Status](https://readthedocs.org/projects/ucc/badge/?version=latest)](https://ucc.readthedocs.io/en/latest/?badge=latest)
[![Discord Chat](https://img.shields.io/badge/dynamic/json?color=blue&label=Discord&query=approximate_presence_count&suffix=%20online.&url=https%3A%2F%2Fdiscord.com%2Fapi%2Finvites%2FJqVGmpkP96%3Fwith_counts%3Dtrue)](http://discord.unitary.foundation)

The goal of this tutorial is to give you an overview of `ucc` features, and provide an easy entry to test `ucc` on your existing quantum programs, and to explore developing custom compiler passes.

Note that `ucc` is in its early stages, building mostly on the shoudlers of other libraries like qiskit and qbraid. This notebook gives you a flavor of where `ucc` is heading, but there's plenty of work ahead to drive utility and adoption. Early feedback is precious to ensuring that direction will be fruitful.

In [ ]:
# Imports used throughout this demo
# Assumes you followed the instructions in the README to setup
# your environment.
import ucc
from ucc import compile

## Quick start

By leveraging [qBraid](https://github.com/qBraid/qBraid), UCC interfaces automatically with multiple quantum computing frameworks, including [Qiskit](https://github.com/Qiskit/qiskit), [Cirq](https://github.com/quantumlib/Cirq), and [PyTKET](https://github.com/CQCL/tket) and supports programs in OpenQASM 2 and [OpenQASM 3](https://openqasm.com/).

The list of supported formats is

In [ ]:
ucc.supported_circuit_formats

Let's take `ucc` compile for a spin with `pytket` ([docs](https://docs.quantinuum.com/tket/user-guide/index.html)) and `cirq` ([docs](https://quantumai.google/cirq)) on a simple 2-qubit circuit.

Starting with `pytket`, let's draw the pre and post compiled-circuits:

In [ ]:

# Using pytket, show the pre and post compiled basic circuit
import pytket
from pytket.circuit.display import render_circuit_jupyter as Tketdraw


tket_circuit = pytket.Circuit(2)
tket_circuit.H(0)
tket_circuit.CX(0, 1)
tket_compiled_circuit = compile(tket_circuit)

Tketdraw(tket_circuit)
Tketdraw(tket_compiled_circuit)

Now let's do the same using the `cirq` library

In [ ]:
import cirq

qubits = cirq.LineQubit.range(2)
cirq_circuit = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.CNOT(qubits[0], qubits[1])
)
cirq_compiled_circuit = compile(cirq_circuit)

print(cirq_circuit)
print("------------------------")
print(cirq_compiled_circuit)

There's a few things to notice from these toy examples:

* The same `compile` function is used to compile each circuit, even though they are different front-end formats.
* This default `compile` has changed to a target gateset with single qubit rotations and controlled-X gates.

Let's step it up a little, and throw a more complex circuit at `ucc`, this time using `qiskit` as the front-end:

In [ ]:
from qiskit.circuit.random import random_clifford_circuit
input_gates = ["cx", "cz", "cy", "swap", "x", "y", "z", "s", "sdg", "h"]
num_qubits = 10
random_circuit = random_clifford_circuit(
    num_qubits, gates=input_gates, num_gates=10 * num_qubits * num_qubits
)
compiled_random_circuit = compile(random_circuit)
print(f"Number of multi-qubit gates in original circuit: {random_circuit.num_nonlocal_gates()}")
print(f"Gates used in original circuit: {random_circuit.count_ops()}")
print(f"Number of multi-qubit gates in compiled circuit: {compiled_random_circuit.num_nonlocal_gates()}")
print(f"Gates used in compiled circuit: {compiled_random_circuit.count_ops()}")

You'll notice a much bigger change in this circuit, but again compiling to the default target gateset.

## Customizing
The above examples highlight the `ucc` design goal of simplicity with reasonable defaults. But what if you want more control? `ucc` provides a few options (with more to come!).

First, `ucc` supports specifying a custom `compiler` class to `compile`. For now, this class should be an instance of the default compiler `UCCDefault1`, which contains a pre-defined set of Qiskit compilation passes. When constriction this class, you can specify a `target_device` to indicate target hardware specific properties that the compiler passes can reference.

### Coupling
For this example, consider a target device with restricted multi-qubit coupling, in this case a linear 4-qubit circuit with only nearest neighbor coupling. Using `cirq` as the front-end:

In [ ]:
from ucc import UCCDefault1

from qiskit.transpiler import Target
from qiskit.circuit.library import CXGate
# Create a simple target that does not have direct CX between 0 and 2
target = Target(description="Fake device", num_qubits=3)
target.add_instruction(CXGate(), {(0, 1): None, (1, 2): None})


qubits = cirq.LineQubit.range(3)
cirq_circuit = cirq.Circuit(
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.CNOT(qubits[0], qubits[2]) # <- 0 <> 2 not natively supported by topology
)

nn_compiler = UCCDefault1(target_device=target)
cirq_compiled_circuit = compile(cirq_circuit, compiler = nn_compiler)

print(cirq_circuit)
print("------------------------")
print(cirq_compiled_circuit)


You can see that the controlled-not gates are now only between nearest neighbors.

### Passes
For compilation strategies, `ucc` uses a default set of `qiskit` compilation passes in the `UCCDefault1` class. These are a set of `qiskit` passes that showed strong performance in benchmarking. 

However, users are free to add their own passes to this set, or develop their own custom sets of passes. Indeed, we expect this to be an active area of `ucc` development. 

For now, an example of adding a new pass is below. This toy pass swaos H for X gates, and is just meant as an example of interacting with `ucc` compilation.

In [ ]:
# Create the new pass
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.circuit.library import HGate, XGate
class HtoX(TransformationPass):
        """Toy transformation that converts all H gates to X gates"""

        def run(self, dag):
            for node in dag.op_nodes():
                if not isinstance(node.op, HGate):
                    continue
                dag.substitute_node(node, XGate())
            return dag

qubits = cirq.LineQubit.range(1)
cirq_circuit = cirq.Circuit(cirq.H(qubits[0]))

# Example usage with a cirq circuit, stil showcasing the cross-frontend compatibility
print("Pre-compile")
print(cirq_circuit)


htox_compiler = UCCDefault1()
htox_compiler.pass_manager.append(HtoX())
post_compiler_circuit = compile(cirq_circuit, compiler=htox_compiler)

print("Post-compile")
print(post_compiler_circuit)

## What next?

1. Take `ucc` `compile` for a spin on one of your circuits. How did it change behavior?
2. [Contribute](https://ucc.readthedocs.io/en/latest/contributing.html#proposing-a-new-transpiler-pass) a custom optimization pass
3. Join the [discussion](https://github.com/unitaryfund/ucc/discussions) on the evolution of `ucc` 

As mentioned in the [launch blogpost](https://unitary.foundation/posts/2025_ucc_launch_blog/), for development priorities post-launch, we want to focus on what we think is most impactful for the progress of quantum computing – and where we have unique expertise at Unitary Foundation – leveraging our diverse and dynamic quantum open-source community to push into the regime of thousands of qubits, 10s of thousands of gates, where compilers require novel architectures and abstractions to handle errors.

Key roadmap items include:

* Quantum Error Mitigation (QEM): Integration with Mitiq, our cross-platform QEM library with 212k+ downloads and 100+ citations.
* Hardware-Aware Compilation: Custom routing and scheduling optimized for emerging architectures.
* Quantum Error Correction (QEC): Implementing early fault tolerance protocols in conjunction with error mitigation
* Hybrid classical-quantum programming: Mid-circuit measurements, fast feedback, repeat-until-success, etc.

Stay tuned for future demo notebooks with these capabilities!